Toxicity

In [ ]:
# Importing required libraries and its utilities
import numpy as np

np.random.seed(123)
import tensorflow as tf

tf.random.set_seed(123)
from deepchem.molnet import load_tox21


In [ ]:
# Tox21 is a part of Deepchem library
# so we can convieniently download it using load_tox21 function
tox21_tasks, tox21_datasets, transformers = deepchem.molnet.load_tox21()
train_dataset, valid_dataset, test_dataset = tox21_datasets


# Define metric for the model
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, 
                           np.mean, 
                           mode="classification")

In [ ]:
tox21_tasks

In [ ]:
train_dataset

In [ ]:
# Define and fit the model
model = deepchem.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])
model.fit(train_dataset, nb_epoch=10)
metric = deepchem.metrics.Metric(deepchem.metrics.roc_auc_score, np.mean)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

In [ ]:
print("Train scores")
print(train_scores)

print("Validation scores")
print(valid_scores)

In [ ]:
model.predict(test_dataset)